<a href="https://colab.research.google.com/github/Dhruv721725/projects/blob/main/p1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 👗 Fashion-MNIST → CNN → TensorFlow Lite (Colab Starter)

This notebook teaches you the full pipeline:
1) Understand the key building blocks (activation, bias, convolution, pooling, etc.)
2) Load the **Fashion-MNIST** dataset (10 clothing classes)
3) Build & train a **CNN** in TensorFlow/Keras
4) Evaluate and visualize performance
5) Convert to **TensorFlow Lite** (with quantization) and test inference

> Tip: Go to **Runtime → Change runtime type → GPU** for faster training.


## 🔎 Concept Cheatsheet (Simple & Practical)

- **Tensor / Shape**: Multi‑dimensional array. For images here: `(batch, 28, 28, 1)`.
- **Bias**: A trainable constant added to each neuron’s weighted sum. Let’s a neuron shift the activation threshold.
- **Activation Function**: Turns the linear sum into a non‑linear output.
  - **ReLU**: `max(0, x)` – fast, avoids vanishing gradients. Most common in CNNs.
  - **LeakyReLU**: Like ReLU but small slope for negatives to avoid dead neurons.
  - **Sigmoid**: Outputs (0,1). Good for binary outputs, not hidden CNN layers.
  - **Tanh**: Outputs (−1,1). Less common in modern CNNs.
  - **Softmax**: Converts logits to probabilities across classes (used in the final layer).
- **Convolution (Conv2D)**: Learns small kernels/filters (e.g., 3×3) that slide over the image to detect patterns.  
  - **Kernel/Filter**: Small learnable matrix. Number of filters = number of feature maps learned.
  - **Stride**: Step size while sliding (usually 1).
  - **Padding**: `"same"` keeps size; `"valid"` shrinks size.
- **Pooling**: Downsamples feature maps (e.g., **MaxPooling2D(2×2)**) to reduce size and help translation invariance.
- **Batch Normalization**: Normalizes activations to stabilize & speed up training.
- **Dropout**: Randomly drops a fraction of activations during training to reduce overfitting.
- **Loss**: What we minimize. For multi‑class classification with integer labels, use **SparseCategoricalCrossentropy**.
- **Optimizer**: How we update weights. **Adam** works well by default.
- **Learning Rate (LR)**: Step size for updates. Too high = diverge, too low = slow. We use a scheduler.
- **Metrics**: Accuracy, confusion matrix to see per‑class performance.
- **Overfitting Control**: Data augmentation, dropout, early stopping, L2 regularization.
- **TFLite**: Lightweight runtime for on‑device inference (works offline). We’ll convert our Keras model to `.tflite`.


In [ ]:
# @title ✅ Setup & Imports (run this first)
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import layers, models
from sklearn.metrics import confusion_matrix, classification_report
import itertools

print("TensorFlow:", tf.__version__)

# GPU check
gpus = tf.config.list_physical_devices('GPU')
print("GPUs available:", gpus)


In [ ]:
# @title Class Names
class_names = [
    "T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
    "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"
]
class_names


In [ ]:
# @title 📥 Load Fashion-MNIST
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# Normalize to [0,1] and add channel dimension
x_train = (x_train / 255.0).astype("float32")[..., np.newaxis]  # (N, 28, 28, 1)
x_test  = (x_test  / 255.0).astype("float32")[..., np.newaxis]

print("Train:", x_train.shape, y_train.shape)
print("Test :", x_test.shape, y_test.shape)


In [ ]:
# @title 👀 Peek at a few samples
plt.figure(figsize=(8,8))
for i in range(16):
    plt.subplot(4,4,i+1)
    plt.imshow(x_train[i].squeeze(), cmap="gray")
    plt.title(class_names[y_train[i]])
    plt.axis("off")
plt.tight_layout()
plt.show()


In [ ]:
# @title 🔀 Train / Validation Split
from sklearn.model_selection import train_test_split

x_tr, x_val, y_tr, y_val = train_test_split(
    x_train, y_train, test_size=0.1, stratify=y_train, random_state=42
)
print("Train:", x_tr.shape, y_tr.shape, "| Val:", x_val.shape, y_val.shape)


In [ ]:
# @title 🔧 tf.data pipelines (+ light augmentation)
batch_size = 128
AUTOTUNE = tf.data.AUTOTUNE

def augment(image, label):
    # For grayscale 28x28, keep augmentations subtle
    image = tf.image.random_flip_left_right(image)
    # Small random brightness/contrast
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.9, upper=1.1)
    return image, label

ds_train = tf.data.Dataset.from_tensor_slices((x_tr, y_tr))\
    .shuffle(10_000)\
    .map(augment, num_parallel_calls=AUTOTUNE)\
    .batch(batch_size)\
    .prefetch(AUTOTUNE)

ds_val = tf.data.Dataset.from_tensor_slices((x_val, y_val))\
    .batch(batch_size)\
    .prefetch(AUTOTUNE)

ds_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))\
    .batch(batch_size)\
    .prefetch(AUTOTUNE)


In [ ]:
# @title 🧠 Build a CNN (Conv → BN → ReLU → Pool → Dropout blocks)
def conv_block(x, filters):
    x = layers.Conv2D(filters, (3,3), padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    return x

inputs = layers.Input(shape=(28,28,1))
x = conv_block(inputs, 32)
x = conv_block(x, 32)
x = layers.MaxPooling2D((2,2))(x)
x = layers.Dropout(0.25)(x)

x = conv_block(x, 64)
x = conv_block(x, 64)
x = layers.MaxPooling2D((2,2))(x)
x = layers.Dropout(0.3)(x)

x = conv_block(x, 128)
x = layers.MaxPooling2D((2,2))(x)
x = layers.Dropout(0.4)(x)

x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.4)(x)
outputs = layers.Dense(10, activation="softmax")(x)  # softmax for class probs

model = models.Model(inputs, outputs)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)
model.summary()


In [ ]:
# @title 🏃 Train with EarlyStopping + LR scheduler
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True, monitor="val_accuracy"),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=2, min_lr=1e-5, monitor="val_loss"),
]

history = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=15,
    callbacks=callbacks
)


In [ ]:
# @title 📈 Plot training curves
hist = history.history
plt.figure()
plt.plot(hist["accuracy"], label="train_acc")
plt.plot(hist["val_accuracy"], label="val_acc")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

plt.figure()
plt.plot(hist["loss"], label="train_loss")
plt.plot(hist["val_loss"], label="val_loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()


In [ ]:
# @title ✅ Evaluate & Confusion Matrix
test_loss, test_acc = model.evaluate(ds_test, verbose=0)
print(f"Test accuracy: {test_acc:.4f}")

y_prob = model.predict(ds_test)
y_pred = np.argmax(y_prob, axis=1)

cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix:\n", cm)

# Plot confusion matrix
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix'):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=(8,8))
    plt.imshow(cm, interpolation='nearest')
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, ha="right")
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()

plot_confusion_matrix(cm, class_names, normalize=False, title="Confusion Matrix (Counts)")
plot_confusion_matrix(cm, class_names, normalize=True, title="Confusion Matrix (Normalized)")

print("\nClassification report:\n")
print(classification_report(y_test, y_pred, target_names=class_names))


In [ ]:
# @title 💾 Save Keras model & labels
model.save("fashion_mnist_cnn.h5")

with open("label_map.txt", "w") as f:
    for i, name in enumerate(class_names):
        f.write(f"{i}:{name}\n")

print("Saved: fashion_mnist_cnn.h5 and label_map.txt")


In [ ]:
# @title 🔄 Convert to TFLite (Dynamic Range Quantization)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # dynamic-range quantization
tflite_model = converter.convert()

with open("fashion_mnist_cnn_dynamic.tflite", "wb") as f:
    f.write(tflite_model)

print("Saved: fashion_mnist_cnn_dynamic.tflite")


In [ ]:
# @title 🔬 Convert to TFLite (Full Integer INT8 Quantization)
def representative_data_gen():
    for i in range(200):  # few hundred samples are enough
        img = x_tr[i]
        img = np.expand_dims(img, axis=0).astype(np.float32)
        yield [img]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

int8_tflite = converter.convert()
with open("fashion_mnist_cnn_int8.tflite", "wb") as f:
    f.write(int8_tflite)

print("Saved: fashion_mnist_cnn_int8.tflite (full integer)")


In [ ]:
# @title 🧪 Test TFLite Inference (Dynamic model)
# This test uses the dynamic-quantized TFLite model.
interpreter = tf.lite.Interpreter(model_path="fashion_mnist_cnn_dynamic.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Pick a test image
idx = 0
img = x_test[idx:idx+1]  # shape (1, 28, 28, 1)

# If input type is not float32, cast accordingly
inp_type = input_details[0]['dtype']
img_for_infer = img.astype(inp_type)

interpreter.set_tensor(input_details[0]['index'], img_for_infer)
interpreter.invoke()
pred = interpreter.get_tensor(output_details[0]['index'])[0]
pred_class = np.argmax(pred)

plt.imshow(img[0].squeeze(), cmap="gray")
plt.title(f"True: {class_names[y_test[idx]]} | Pred: {class_names[pred_class]}")
plt.axis("off")
plt.show()


## 📦 (Optional) Collecting Your Own Fashion Data
If you want to extend/retrain with custom clothing images, use this folder layout:
```
dataset/
  train/
    T-shirt_top/     img001.jpg ...
    Trouser/
    Pullover/
    Dress/
    Coat/
    Sandal/
    Shirt/
    Sneaker/
    Bag/
    Ankle_boot/
  val/
    T-shirt_top/
    ...
  test/
    ...
```
Tips:
- Keep **backgrounds simple** and **lighting consistent**.
- Aim for **600–1,000 images per class** to start (you can grow later).
- Use **data augmentation** during training (flips, brightness, contrast).
- Ensure **no overlap** between the same **physical items** across train/val/test.
- If images are RGB and larger, you can resize to 64×64 or 96×96 and adjust the CNN input shape.
